# Teaming - IS843


The seed below ensures that the results are random, but reproducible!

In [ ]:
seed = 123

Reading the data (data is kept private for students' privacy)

In [ ]:
students = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load('gs://is843/data/students-is843-spring-2021.csv')

students.show(19)

Leads were selected in the class:

In [ ]:
from pyspark.sql.functions import *

leads = students.where("team").select("name")

print("after shuffling:")
leads = leads.orderBy(rand(seed))

leads.show()

In [ ]:
for i in range(4):
    take_n = 4
    group = students.where("!team").select("name").orderBy(rand(seed)).take(take_n)
    print("\n", "Team ", i+1, ": ", sep="")
    group_l = [x[0] for x in group]
    students = students.withColumn("team", when(col("name").isin(group_l), True).otherwise(col("team")))
    print(leads.collect()[i][0])
    print(*group_l, sep = "\n")
    print("\n", "-----------------------------------", sep = "")